In [1]:
import pandas as pd
import numpy as np

In [22]:
training_data = pd.read_csv("trainingSet.csv")
features = [col for col in training_data.columns if col not in ['decision']]

gender             age           age_o  race  race_o  samerace  \
0       0  (17.999, 26.0]    (26.0, 34.0]     0       2         0   
1       0  (17.999, 26.0]  (17.999, 26.0]     0       2         0   
2       0  (17.999, 26.0]  (17.999, 26.0]     0       2         0   
3       0  (17.999, 26.0]  (17.999, 26.0]     0       3         0   
4       0  (17.999, 26.0]  (17.999, 26.0]     0       2         0   

  importance_same_race importance_same_religion  field pref_o_attractive  ...  \
0        (-0.001, 2.0]               (2.0, 4.0]    121        (0.2, 0.4]  ...   
1        (-0.001, 2.0]               (2.0, 4.0]    121        (0.4, 0.6]  ...   
2        (-0.001, 2.0]               (2.0, 4.0]    121        (0.2, 0.4]  ...   
3        (-0.001, 2.0]               (2.0, 4.0]    121        (0.2, 0.4]  ...   
4        (-0.001, 2.0]               (2.0, 4.0]    121        (0.4, 0.6]  ...   

         theater       movies     concerts        music    shopping  \
0  (-0.001, 2.0]  (8.0, 10.0]  (8.0, 10.0]  (8.0, 10.0]  (6.0, 8.0]   
1  (-0.001, 2.0]  (8.0, 10.0]  (8.0, 10.0]  (8.0, 10.0]  (6.0, 8.0]   
2  (-0.001, 2.0]  (8.0, 10.0]  (8.0, 10.0]  (8.0, 10.0]  (6.0, 8.0]   
3  (-0.001, 2.0]  (8.0, 10.0]  (8.0, 10.0]  (8.0, 10.0]  (6.0, 8.0]   
4  (-0.001, 2.0]  (8.0, 10.0]  (8.0, 10.0]  (8.0, 10.0]  (6.0, 8.0]   

            yoga interests_correlate expected_happy_with_sd_people  \
0  (-0.001, 2.0]         (-0.2, 0.2]                    (2.0, 4.0]   
1  (-0.001, 2.0]          (0.2, 0.6]                    (2.0, 4.0]   
2  (-0.001, 2.0]          (0.6, 1.0]                    (2.0, 4.0]   
3  (-0.001, 2.0]          (0.2, 0.6]                    (2.0, 4.0]   
4  (-0.001, 2.0]          (0.2, 0.6]                    (2.0, 4.0]   

         like decision  
0  (6.0, 8.0]        1  
1  (6.0, 8.0]        1  
2  (6.0, 8.0]        1  
3  (4.0, 6.0]        1  
4  (4.0, 6.0]        0  

[5 rows x 53 columns]

In [5]:
def nbc(t_frac):
    training_sample = training_data.sample(frac=t_frac, random_state=47)
    p_decision = training_sample.groupby(['decision'])['decision'].count()/len(training_sample)
    p_attributes = {}
    for col in features:
        p_attributes[col] = (training_sample.groupby(['decision', col])['decision'].count().unstack(fill_value=0).stack() + 1)/(training_sample.groupby(['decision'])['decision'].count()+len(training_sample[col].sort_values().unique()))
    return p_decision, p_attributes

In [6]:
p_decision, p_attributes = nbc(1)

In [7]:
def get_class(x):
    p_estimated_decision = {}
    for val in training_data['decision'].unique():
        p_estimated_decision[val] = p_decision[val]
        for col in features:
            if x[col] in p_attributes[col][val]:
                p_estimated_decision[val] = p_estimated_decision[val] *p_attributes[col][val][x[col]]
    return p_estimated_decision

In [61]:
test_data = pd.read_csv("testSet.csv")
accuracy_test = 0
for i,x in test_data.iterrows():
    p_estimated_decision = get_class(x)
    accuracy_test += 1 if max(p_estimated_decision, key=p_estimated_decision.get) == x['decision'] else 0

In [62]:
print("Testing Accuracy:", accuracy_test/len(test_data))

0.7494440326167532
0.7494440326167532


In [63]:
training_data = pd.read_csv("trainingSet.csv")
accuracy_training = 0
for i,x in training_data.iterrows():
    p_estimated_decision = get_class(x)
    accuracy_training += 1 if max(p_estimated_decision, key=p_estimated_decision.get) == x['decision'] else 0

In [64]:
print("Training Accuracy:", accuracy_test/len(test_data))

0.77516218721038
0.77516218721038
